Import libraries

In [14]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

Get the page links

In [15]:
url = "http://www.squarepharma.com.bd/products-by-generic.php?type=pharma&char=A"
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

link_list=[chr(i) for i in range(ord('A'), ord('Z')+1)]

pages = [i.text for i in soup.find_all('a') if i.text in link_list]
print(pages)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Z']


Crawling the web pages for gathering the generic name of the medicine.

In [16]:
generic=[]
name=[]
description=[]
indication =[]
dosage =[]
preparation = []
for cp in range(len(pages)):
    url = "http://www.squarepharma.com.bd/products-by-generic.php?type=pharma&char=" + pages[0]
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html')
    divs = soup.findAll('div', attrs={'class' : 'col-xs-12 col-sm-4 col-md-4 col-lg-4'})
    links = []
    for div in divs:
        if 'product-details.php?pid=' in div.a['href']:
            links.append(div.a['href'])
    divs = [i.text for i in soup.find_all(class_='col-xs-12 col-sm-4 col-md-4 col-lg-4')]

    for i in range(len(divs)):
        descriptions = divs[i].split("\n")
        descriptions = [descrip for descrip in descriptions if (len (descrip) > 0 and descrip!='Find More') ]
        
        if len(descriptions)==3:
            generic.append(descriptions[0])
            name.append(descriptions[1])
            description.append(descriptions[2])

            url = "http://www.squarepharma.com.bd/" + links[i]
            html = urlopen(url)
            soup = BeautifulSoup(html, 'html')
            info = [i.text for i in soup.findAll('t')]
            if len(info)>=1:
                indication.append(info[0])
            if len(info)>=2:
                dosage.append(info[1])
            else:
                dosage.append(None)
            if len(info)>=3:
                preparation.append(info[2])
            else:
                preparation.append(None)
            

print ("Done!")

Flexi®
Done!


Save the info in a CSV

In [ ]:
cdata = {"Generic":generic,"Name":name,"Description":description,"Indication":indication,"Dosage":dosage,"Preparation":preparation}
df = pd.DataFrame(data = cdata)
df.to_csv("medicine.csv", header=True, index=False)